Note: This is very similar to the StreamToParquet.ipynb, except in this one we will add an additional group column  which will be used to categories the tweets based on the subjects we are tracking, see twitter_kafka_producer.py.

Import necassary libraries:

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, LongType, BooleanType
from IPython.display import display, clear_output
import time

Setup connection and subscribed to the twitterdata topic:

In [2]:
# Open spark session
spark = SparkSession.builder \
        .appName('kafka') \
        .getOrCreate()

# Subscribed to twitterdata topic
stream_df = spark \
  .readStream \
  .format("kafka") \
  .option("kafka.bootstrap.servers", "broker:29092") \
  .option("startingOffsets", "earliest") \
  .option("subscribe", "twitterdata") \
  .load()

Convert the value column to string, then convert the JSON string to StructType *(online including the columns we are interested in)*:

In [3]:
string_stream_df = stream_df.withColumn("value", stream_df["value"].cast(StringType()))
tweet_schema = StructType([
    StructField('created_at', StringType(), True),
    StructField('id', LongType(), True),
    StructField('text', StringType(), True),
    StructField('is_quote_status', BooleanType(), True),
    StructField('in_reply_to_user_id', LongType(), True),
    StructField('user', StructType([
        StructField('id', LongType(), True),
        StructField('followers_count', IntegerType(), True),
        StructField('friends_count', IntegerType(), True),
        StructField('created_at', StringType(), True)
    ])),
    StructField('extended_tweet', StructType([
        StructField('full_text', StringType(), True)
    ])),
    StructField('retweeted_status', StructType([
        StructField('id', LongType(), True),
        StructField('extended_tweet', StructType([
            StructField('full_text', StringType(), True)
        ]))
    ])),
    StructField('retweet_count', IntegerType(), True),
    StructField('favorite_count', IntegerType(), True),
    StructField('quote_count', IntegerType(), True),
    StructField('reply_count', IntegerType(), True)
])
struct_stream_df = string_stream_df.withColumn("value", F.from_json("value", tweet_schema))

The text column will not always contain the relevant data for example quotes and retweets will usually only contains a truncated version of the orginal tweet and therfore may not contain the keyword we are tracking, for this reason to generate the *category* column we will concatinate the *text* and *full_text's* columns from extended_tweet & retweet_status then categorise based on keyword found in the new *combined_text* column.

In [4]:
def get_pattern(value):
    return f'(?i)({value})'

twitter_flat_df = struct_stream_df.select(
    'timestamp'
    , 'value.text'
    , F.col('value.retweeted_status.extended_tweet.full_text').alias('retweet_extended_full_text')
    , F.col('value.extended_tweet.full_text').alias('extended_full_text')
).withColumn(
    'combined_text'
    , F.concat_ws(' ', F.col('text'), F.col('retweet_extended_full_text'), F.col('extended_full_text'))
).withColumn(
    'category'
    , F.when(F.col('combined_text').rlike(get_pattern('cryptocurrency')), 'cryptocurrency') \
        .when(F.col('combined_text').rlike(get_pattern('crypto')), 'crypto') \
        .when(F.col('combined_text').rlike(get_pattern('binance')), 'binance') \
        .when(F.col('combined_text').rlike(get_pattern('coinbase')), 'coinbase') \
        .when(F.col('combined_text').rlike(get_pattern('coinmarketcap')), 'coinmarketcap') \
        .when(F.col('combined_text').rlike(get_pattern('musk')), 'musk') \
        .when(F.col('combined_text').rlike(get_pattern('memecoin')), 'memecoin') \
        .when(F.col('combined_text').rlike(get_pattern('shitcoin')), 'shitcoin') \
        .when(F.col('combined_text').rlike(get_pattern('moon')), 'moon') \
        .when(F.col('combined_text').rlike(get_pattern('hodl')), 'hodl') \
        .when(F.col('combined_text').rlike(get_pattern('fud')), 'fud') \
        .when(F.col('combined_text').rlike(get_pattern('bitcoin')), 'bitcoin') \
        .when(F.col('combined_text').rlike(get_pattern('btc')), 'btc') \
        .when(F.col('combined_text').rlike(get_pattern('ethereum')), 'ethereum') \
        .when(F.col('combined_text').rlike(get_pattern('ether')), 'ether') \
        .when(F.col('combined_text').rlike(get_pattern('gwei')), 'gwei') \
        .when(F.col('combined_text').rlike(get_pattern('vitalik buterin')), 'vitalik buterin') \
        .when(F.col('combined_text').rlike(get_pattern('gavin wood')), 'gavin wood') \
        .when(F.col('combined_text').rlike(get_pattern('erc20')), 'erc20') \
        .when(F.col('combined_text').rlike(get_pattern('dogecoin')), 'dogecoin') \
        .when(F.col('combined_text').rlike(get_pattern('doge')), 'doge') \
        .when(F.col('combined_text').rlike(get_pattern('billy markus')), 'billy markus') \
        .when(F.col('combined_text').rlike(get_pattern('jackson palmer')), 'jackson palmer') \
        .when(F.col('combined_text').rlike(get_pattern('pancakeswap')), 'pancakeswap') \
        .when(F.col('combined_text').rlike(get_pattern('cake')), 'cake') \
        .when(F.col('combined_text').rlike(get_pattern('swap')), 'swap') \
        .when(F.col('combined_text').rlike(get_pattern('eth')), 'eth')
    )


Quick sanity check make sure things are being categorised:

In [6]:
twitter_flat_stream = twitter_flat_df.writeStream.format('memory').queryName('twitter_flat').start()

In [9]:
clear_output(wait=True)
display(spark.sql(
    """
    SELECT 
        t.timestamp
        , t.combined_text
        , t.category 
    FROM twitter_flat t
    """
).show(20))
time.sleep(1)

+--------------------+--------------------+--------+
|           timestamp|       combined_text|category|
+--------------------+--------------------+--------+
|2021-06-08 10:25:...|RT @CardanoFeed: ...|ethereum|
|2021-06-08 10:25:...|RT @markjburns88:...|  crypto|
|2021-06-08 10:25:...|My lovely buyer w...|  crypto|
|2021-06-08 10:25:...|RT @Crypt0kyuubi:...|    hodl|
|2021-06-08 10:25:...|RT @Deeeerin: - R...| bitcoin|
|2021-06-08 10:25:...|Dogecoin Legacy s...|dogecoin|
|2021-06-08 10:25:...|@hans_emerson @Cr...|    moon|
|2021-06-08 10:25:...|        To the moon!|    moon|
|2021-06-08 10:25:...|RT @toolzbeib: Th...|    moon|
|2021-06-08 10:25:...|RT @hippiechikmar...|    moon|
|2021-06-08 10:25:...|RT @blockchainrem...| bitcoin|
|2021-06-08 10:25:...|RT @angry_coin: “...|  crypto|
|2021-06-08 10:25:...|@HukAleksandra ht...| bitcoin|
|2021-06-08 10:25:...|RT @mskvsk: 22,55...| bitcoin|
|2021-06-08 10:25:...|RT @NewWorld_Orde...| bitcoin|
|2021-06-08 10:25:...|@SPYJared nobody ...|   

None

Looks good, stop the stream:

In [10]:
twitter_flat_stream.stop()

Here we are defining a windowed aggregation query to which will calculate the number of tweets coming in for each category in a 60 seconds time windows, updating every 10 seconds and with a late threshold of 15 seconds:

In [11]:
window_duration = '60 seconds'
slide_duration = '30 seconds'
late_threshold = '15 seconds'

window_df = twitter_flat_df.withWatermark(
    'timestamp'
    , late_threshold
    ).groupby(
        F.window(twitter_flat_df.timestamp, window_duration, slide_duration)
        , twitter_flat_df.category
    ).count()

To sink this to parquet we need to do it in batches using the foreachBatch callback method, this will be executed once the time window has finshed, we can then use sink the batch of aggregations to parquet.

Ran into a number of issues trying to get his to work, because its a windowed aggregated streaming data frame spark want let you just write it to parquet after a fair amount of googling I came across the following article which demonstrated persisting aggregated stream to a database table: https://docs.databricks.com/spark/latest/structured-streaming/examples.html

Some of the issues encountered:
 - AnalysisException: Data source parquet does not support Complete output mode.
 - overwrite output mode no supported

In [15]:

def write_batch(df, epochId):
    df.write.parquet('data/twitter_windowed_sink.parquet', mode='overwrite')

flat_stream_out = window_df \
    .writeStream \
    .foreachBatch(write_batch) \
    .outputMode("complete") \
    .start()


Let that run for a bit and then stop the streams:

In [16]:
count_stream.stop()
flat_stream_out.stop()

Now lets take a quick look at the data:

In [17]:
df = spark.read.parquet('data/twitter_windowed_sink.parquet')
df.show()

+--------------------+--------------+-----+
|              window|      category|count|
+--------------------+--------------+-----+
|{2021-06-06 10:51...|cryptocurrency|    2|
|{2021-06-06 09:26...|           eth|    1|
|{2021-06-06 10:24...|       binance|    3|
|{2021-06-06 08:19...|cryptocurrency|    2|
|{2021-06-06 07:46...|          hodl|    2|
|{2021-06-06 09:20...|       bitcoin|    3|
|{2021-06-06 09:45...|           btc|    2|
|{2021-06-06 07:17...|          doge|    3|
|{2021-06-06 08:10...|       binance|    6|
|{2021-06-06 09:59...|          hodl|    2|
|{2021-06-06 09:45...|           eth|    2|
|{2021-06-06 08:36...|         ether|    1|
|{2021-06-06 10:40...|       bitcoin|    3|
|{2021-06-06 07:26...|cryptocurrency|    7|
|{2021-06-06 09:59...|          swap|    1|
|{2021-06-06 07:44...|          null|   14|
|{2021-06-06 09:10...|        crypto|   11|
|{2021-06-06 10:02...|           eth|    1|
|{2021-06-06 07:03...|cryptocurrency|    2|
|{2021-06-06 07:13...|      ethe

Close the spark session:

In [11]:
spark.stop()